<a href="https://colab.research.google.com/github/danielaterra/ifmg_topicos_SI/blob/main/TSP_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###  Problema do Caixeiro Viajante
Solução baseada nos códigos disponíveis em: http://www.decom.ufop.br/prof/marcone/Disciplinas/InteligenciaComputacional/Aula%20IC%2018-09-2003.zip

**Profa. Daniela Costa Terra**

In [1]:
import sys
import numpy as np  # importa o pacote com 'alias' ou apelido de 'np'
import math
import matplotlib.pyplot as plt    # o módulo pyplot, parte do matplotlib, como plt
np.random.seed(1000)    ## produz o mesmos numeros 'pseudo'aleatórios a cada execução

In [ ]:
from google.colab import files
uploaded = files.upload()    ## faça upload dos arquivos disponibilizados

Saving C50.TXT to C50.TXT
Saving c50-BestRoute.txt to c50-BestRoute.txt
Saving C50INFO.TXT to C50INFO.TXT


In [29]:
#TODO: def randomicodescida(n, s, matriz_dist, fo, MaxIter=100):

#     print(" ********** Falta implementar ************* \n")

#     print("-------------------------------------------------\n")
#     print("Melhor solucao encontrada pelo Metodo Randomico de descida:\n")
#     print("Funcao objetivo: %d\n", fo)
#     print("Peso da mochila: %d    Inviabilidade = ... \n",   )
#     print("-------------------------------------------------\n")

#TODO: def descida_fi(n, s, matriz_dist, fo):

#     print(" ********** Falta implementar ************* \n")

#     print("-------------------------------------------------\n")
#     print("Melhor solucao encontrada pelo Metodo da descida - first improvment:\n")
#     print("Funcao objetivo: %d\n", fo)
#     print("Peso da mochila: %d    Inviabilidade = ... \n",   )
#     print("-------------------------------------------------\n")


# Heurística de refinamento best improvment (Descida)
def descida_bi(n, s, fo, matriz_dist):
    melhor_fo = -1*np.sum(matriz_dist)
    while (melhor_fo < fo):
          melhor_fo = np.sum(matriz_dist)
          ## Perturba soluçao atual:
          for i in np.arange(n-1):
              cid_i = i
              for j in np.arange(i+1, n):
                  s_linha = s
                  s_linha[i] = s[j]
                  s_linha[j] = cid_i
                  fo_s_linha = calcula_fo(s_linha, matriz_dist)
                  if (fo_s_linha < melhor_fo):
                      melhor_fo = fo_s_linha
                      melhor_troca = (cid_i, j)

          if (melhor_fo < fo):
            #Melhor solucao encontrada
            cid_i = melhor_troca[0]
            s[melhor_troca[0]] = melhor_troca[1]
            s[melhor_troca[1]] = cid_i
            fo = melhor_fo
            melhor_fo = -1*np.sum(matriz_dist)  ## para continuar busca

    return s, calcula_fo(s, matriz_dist)
    ## descida_bi


In [30]:
def le_arq_vetor(arquivo, dtype = int):
    v = []
    with open(arquivo, "r") as arquivo:
      linhas = arquivo.readlines()
      #print(linhas)
    for linha in linhas:
        aux = linha.split(' ')
        for num in aux:
           if dtype == int:
               v.append(int(num))
           else:
               v.append(float(num))
    return np.array(v)


# Constroi matriz quadrada (simétrica) de distâncias entre cidades
def constroi_matriz_distancias(n, locals):
    matriz_dist = np.zeros((n,n))
    for i in range(n):
      cidi = (locals[i,1],locals[i,2])
      for j in range(i+1, n):
          cidj = (locals[j,1],locals[j,2])
          matriz_dist[i,j] = math.dist(cidi, cidj)
          matriz_dist[j,i] = matriz_dist[i,j]
    return matriz_dist


# TODO: constroi uma solucao inicial aleatoria */
def constroi_solucao_inicial_aleatoria(n, s, partida, matriz_dist):
    return None

# Constroi solução gulosa inicial gulosa (cidade mais próxima da atual)
def constroi_solucao_inicial_gulosa(n, matriz_dist, partida=0):
    s = np.zeros(matriz_dist.shape[-1])   # vetor solução do tamanho do número de cidades
    cid_atual = partida
    list_rota = [cid_atual]
    fo = 0
    for i in np.arange(1, n): #numero de cidades - 1
          idxs_dist_crescente = np.argsort(matriz_dist[cid_atual,:])
          for cid in idxs_dist_crescente:
              if (cid not in list_rota) and (cid != cid_atual):
                   list_rota.append(cid)
                   s[cid] = i
                   fo+= matriz_dist[cid_atual, cid]
                   cid_atual = cid
                   break  # primeira cidade de menor distancia
    fo+= matriz_dist[cid_atual, partida]
    return (s, fo)

# Escreve rota da solução
def escreve_rota(s):
  rota = [str(i) + "-" for i in np.argsort(s)]
  rota.append(str(int(s[0])))
  return rota

# Calcula função objetivo
def calcula_fo(s, matriz_dist):
    fo = 0
    idx_rota_ordenado = np.argsort(s)
    for i in np.arange(s.shape[-1]-1):   #até a penúltima cidade
        fo+= matriz_dist[idx_rota_ordenado[i], idx_rota_ordenado[i+1]]
    fo+=matriz_dist[idx_rota_ordenado[-1], idx_rota_ordenado[0]]
    return fo

## inverte bit
def invert_bit(bit):
   if (bit):
      return 0
   else:
      return 1

In [34]:
## Inicializações
MaxIter = 100   # número máximo de iterações para busca de soluções aleatórios

# Lê arquivos para o problema do caixeiro viajange
## Formato C..INFO.txt: n_cidades,  fo_solução_otima (menor distancia)
dados = le_arq_vetor("/content/C50INFO.TXT", dtype=float)     #arq de informações sobre o no. de cidades e solução ótima (em distancia)
n = int(dados[0])
fo_better = float(dados[1])

## Formado C..txt:
# num_cidade   X   Y  (onde X,Y são as coordenadas de localização para cada cidade)
locals = le_arq_vetor("/content/C50.TXT", dtype = float)
locals = locals.reshape((n,3))

n = 5 #teste

## constroi matriz de distâncias
matriz_dist = constroi_matriz_distancias(n, locals)

#penalidade =  calcula_penalidade() ##

# variáveis p/ cálculo da solução
s = np.zeros(n)                       ## solução corrente
s_star = np.zeros(n)                  ## melhor solução
fo = None                             ## funcao objetivo corrente
fo_star = None                        ## melhor funcao objetivo
inv = 0                              ## quantidade de inviabilidade da solucao corrente

print('Comprimento da rota ótima: ', fo_better)
print('Formato da matriz de distâncias: ', matriz_dist.shape)

Comprimento da rota ótima:  429.98
Formato da matriz de distâncias:  (5, 5)


## Solução inicial

In [36]:
partida = 3
s, fo = constroi_solucao_inicial_gulosa(n, matriz_dist)

print("\n-------------------------------------------------")
print("Solução inicial  gulosa: ",  *escreve_rota(s))
print("Funcao objetivo (custo - comprimento do caminho): ", fo)
print("-------------------------------------------------\n")


-------------------------------------------------
Solução inicial  gulosa:  0- 1- 2- 3- 4- 0
Funcao objetivo (custo - comprimento do caminho):  108.44243933466328
-------------------------------------------------



## Heurísticas de refinamento: método da descida
* descida - best improvement
* first improvement
* descida aleatória


In [37]:
 descida_bi(n, s, fo, matriz_dist)

(array([4., 0., 1., 2., 3.]), np.float64(108.44243933466329))

#### Atividades de fixação (avaliada):

1. Corrija o erro em calcula_bi e altere a função gera_melhor_vizinho para fazer o cálculo de maneira inteligente para s' gerada.

2. A implementação da lógica para da descida first improvement?

3. Implemente o código das funções abaixo:
 * Construção de solução inicial aleatoria

4. Implemente a perturbação e o refinamento aleatório para uma solução inicial do problema

5. Compare tempos de execução para cada um dos experimentos abaixo:
- Solução inicial gulosa e refinamento Best Improvement
- Solução inicial gulosa e refinamento Fist Improvement
- Solução inicial aleatópria e Best Improvement

6. Em um gráfico plote os tempos de execução para cada um dos experimentos da questão 4. Use o mesmo método para plotar os pontos relativos ao custo da solução final encontrada.

Compare suas respostas com a melhor solução para o TSP com 50 cidades (arquivo disponível)



